In [1]:
import pandas as pd
import numpy as np
import random
from multiprocessing import Process, Manager
from itertools import repeat
from tqdm import trange

In [2]:
data = pd.read_csv('../input/mlbplaybyplay2010s/all2018.csv', low_memory=False)

In [3]:
# 타석 데이터만 남기기.
pa_events = [2, 3, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
data = data[data['EVENT_CD'].isin(pa_events)].reset_index(drop=True)

# Transition matrix 구하기

In [4]:
# Add Columns.
data['BASE1_RUN_FL'] = data['BASE1_RUN_ID'].notna()
data['BASE2_RUN_FL'] = data['BASE2_RUN_ID'].notna()
data['BASE3_RUN_FL'] = data['BASE3_RUN_ID'].notna()
data['HOME_TEAM_ID'] = data.apply(lambda row: row.GAME_ID[:3], axis=1)

# 초반, 후반 데이터 나누기.
n_games = sum(data.GAME_NEW_FL == 'T')  # 게임 수
train = data[data.GAME_ID.isin(data.GAME_ID.unique()[:n_games//2])]  # 전반기
test = data[data.GAME_ID.isin(data.GAME_ID.unique()[n_games//2:])]  # 후반기

In [5]:
# state_action_cnt: 한 state에서 어떤 action을 하는 경우의 수
state_action_cnt = train.groupby(['OUTS_CT', 'BASE1_RUN_FL', 'BASE2_RUN_FL', 'BASE3_RUN_FL'])['EVENT_CD'].value_counts()

# result_cnt: 한 (state, action)에서 어떤 result로 가는 경우의 수
result_cnt = pd.pivot_table(train, values='EVENT_ID', index=['OUTS_CT', 'BASE1_RUN_FL', 'BASE2_RUN_FL', 'BASE3_RUN_FL', 'EVENT_CD'], columns=['EVENT_OUTS_CT', 'BAT_DEST_ID', 'RUN1_DEST_ID', 'RUN2_DEST_ID', 'RUN3_DEST_ID'], aggfunc='count').fillna(0.0).astype(int)

# trans_mat: 한 (state, action)에서 어떤 result로 갈 확률
trans_mat = result_cnt.apply(lambda x: x / state_action_cnt[result_cnt.index])

# 타자, 투수 policy

In [6]:
min_pa = 100  # 최소 타석 수.

In [7]:
# 초기화
bat = {}
for bat_id in train.BAT_ID.unique():
    bat[bat_id] = {key: 0 for key in pa_events + ['pa']}
    
# 각 타자의 event 개수 세기.
for index, row in train.iterrows():
    bat_id = row['BAT_ID']
    event = row['EVENT_CD']
    bat[bat_id]['pa'] += 1
    bat[bat_id][event] += 1
    
# 평균 타자 기록 (pa < min_pa)
bat['average'] = {key: 0 for key in pa_events + ['pa']}
for bat_id in bat:
    bat['average']['pa'] += bat[bat_id]['pa']
    for event in pa_events:
        bat['average'][event] += bat[bat_id][event]

In [8]:
# 초기화
pit = {}
for pit_id in train.PIT_ID.unique():
    pit[pit_id] = {key: 0 for key in pa_events + ['pa']}
    
# 각 투수의 event 개수 세기.
for index, row in train.iterrows():
    pit_id = row['PIT_ID']
    event = row['EVENT_CD']
    pit[pit_id]['pa'] += 1
    pit[pit_id][event] += 1
    
# 평균 투수 기록 (pa < min_pa)
pit['average'] = {key: 0 for key in pa_events + ['pa']}
for pit_id in pit:
    pit['average']['pa'] += pit[pit_id]['pa']
    for event in pa_events:
        pit['average'][event] += pit[pit_id][event]

# 각 타석에서 각 event가 일어날 확률을 학습

In [9]:
# event_dict 초기화.
event_dict = {}
for event in pa_events:
    event_dict['BAT_'+str(event)] = []
    event_dict['PIT_'+str(event)] = []
    
other_features = {'OUTS_CT', 'BASE1_RUN_FL', 'BASE2_RUN_FL', 'BASE3_RUN_FL', 'EVENT_CD'}
for feature in other_features:
    event_dict[feature] = []

# 각 타석에서 타자 policy와 투수 policy를 가져옴.
for index, row in train.iterrows():
    bat_id = row['BAT_ID']
    pit_id = row['PIT_ID']
    
    for event in pa_events:
        if bat[bat_id]['pa'] >= min_pa:
            event_dict['BAT_'+str(event)].append(bat[bat_id][event] / bat[bat_id]['pa'])
        else:
            event_dict['BAT_'+str(event)].append(bat['average'][event] / bat['average']['pa'])
        
        if pit[pit_id]['pa'] >= min_pa:
            event_dict['PIT_'+str(event)].append(pit[pit_id][event] / pit[pit_id]['pa'])
        else:
            event_dict['PIT_'+str(event)].append(pit['average'][event] / pit['average']['pa'])
            
    for feature in other_features:
        event_dict[feature].append(row[feature])

event_df = pd.DataFrame(event_dict)

In [10]:
X = event_df.drop('EVENT_CD', axis=1)
y = event_df['EVENT_CD']

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', n_jobs=-1)
model.fit(X, y)
# pd.DataFrame(model.predict_proba(X), columns=pa_events).mean()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# 각 팀별 평균 수비력

In [11]:
away_event_cnt = train.groupby('AWAY_TEAM_ID')['EVENT_CD'].value_counts()
home_event_cnt = train.groupby('HOME_TEAM_ID')['EVENT_CD'].value_counts()

team_info = {}
for team, event in away_event_cnt.index:
    if team not in team_info:
        team_info[team] = {event: 0 for event in pa_events + ['pa']}
    team_info[team][event] += away_event_cnt[(team, event)]
    team_info[team]['pa'] += away_event_cnt[(team, event)]
for team, event in home_event_cnt.index:
    if team not in team_info:
        team_info[team] = {event: 0 for event in pa_events + ['pa']}
    team_info[team][event] += home_event_cnt[(team, event)]
    team_info[team]['pa'] += home_event_cnt[(team, event)]

In [12]:
# (action, proba)를 dictionary 형태로 받아서 random action을 선택하는 함수
def choose_rand_action(probas):
    probas = dict(probas)  # {2: 0.45, 3: 0.22, ...}
    choice = random.uniform(0, 1)
    accu = 0
    for action, proba in probas.items():
        accu += proba
        if choice < accu:
            return action
    return action

In [13]:
def get_policy(bat_id, pit_id, state, pit_team, sec_pit=False):
    features = []
    for event in pa_events:
        if bat_id in bat and bat[bat_id]['pa'] >= min_pa:
            features.append(bat[bat_id][event] / bat[bat_id]['pa'])
        else:
            features.append(bat['average'][event] / bat['average']['pa'])
        
        if not sec_pit:  # 선발 투수인 경우.
            if pit_id in pit and pit[pit_id]['pa'] >= min_pa:
                features.append(pit[pit_id][event] / pit[pit_id]['pa'])
            else:
                features.append(pit['average'][event] / pit['average']['pa'])
        else:  # 선발 투수가 아닌 경우.
            if pit_id in pit:
                proba = (team_info[pit_team][event] - pit[pit_id][event]) / (team_info[pit_team]['pa'] - pit[pit_id]['pa'])
            else:
                proba = team_info[pit_team][event] / team_info[pit_team]['pa']
            features.append(proba)
            
    features = features + state
    
    probas = model.predict_proba([features])
    return {event: probas[0, index] for index, event in enumerate(pa_events)}

# Simulation

In [14]:
n_simuls = 250
n_procs = 25

In [15]:
def simulation(away_scores, home_scores, case_num):
    state = {
        'INN_CT': 1,
        'BAT_HOME_ID': 0,
        'OUTS_CT': 0,
        'BASE1_RUN_FL': False,
        'BASE2_RUN_FL': False,
        'BASE3_RUN_FL': False,
        'AWAY_BAT_CD': 1,
        'HOME_BAT_CD': 1,
        'AWAY_SCORE_CT': 0,
        'HOME_SCORE_CT': 0,
        'END_FL': False
    }

    while state['END_FL'] == False:  # Until the game ends.
        if state['BAT_HOME_ID'] == 0:  # away team batting
            policy = get_policy(away_batters[state['AWAY_BAT_CD']-1], home_pitcher,
                                [state['OUTS_CT'], state['BASE1_RUN_FL'], state['BASE2_RUN_FL'], state['BASE3_RUN_FL']],
                                home_team, state['INN_CT'] >= 6)
        else:  # home team batting
            policy = get_policy(home_batters[state['HOME_BAT_CD']-1], away_pitcher,
                                [state['OUTS_CT'], state['BASE1_RUN_FL'], state['BASE2_RUN_FL'], state['BASE3_RUN_FL']],
                                away_team, state['INN_CT'] >= 6)

        action = choose_rand_action(policy)

        state_action = (state['OUTS_CT'], state['BASE1_RUN_FL'], state['BASE2_RUN_FL'], state['BASE3_RUN_FL'], action)

        if state_action not in trans_mat.index:
            continue

        trans_probas = trans_mat.loc[state_action]
        EVENT_OUTS_CT, BAT_DEST_ID, RUN1_DEST_ID, RUN2_DEST_ID, RUN3_DEST_ID = choose_rand_action(trans_probas)

        state['OUTS_CT'] += EVENT_OUTS_CT

        base = [0, 0, 0, 0, 0, 0, 0]
        for runner in [BAT_DEST_ID, RUN1_DEST_ID, RUN2_DEST_ID, RUN3_DEST_ID]:
            base[runner] += 1

        # 득점 계산
        runs_scored = base[4] + base[5] + base[6]

        # 주자 갱신
        state['BASE1_RUN_FL'] = True if base[1] > 0 else False
        state['BASE2_RUN_FL'] = True if base[2] > 0 else False
        state['BASE3_RUN_FL'] = True if base[3] > 0 else False

        if state['BAT_HOME_ID'] == 0:  # away team 공격이면
            state['AWAY_BAT_CD'] = state['AWAY_BAT_CD'] % 9 + 1
            state['AWAY_SCORE_CT'] += runs_scored
        else:  # home team 공격
            state['HOME_BAT_CD'] = state['HOME_BAT_CD'] % 9 + 1
            state['HOME_SCORE_CT'] += runs_scored

        if state['OUTS_CT'] >= 3:
            state['OUTS_CT'] = 0
            state['BASE1_RUN_FL'] = False
            state['BASE2_RUN_FL'] = False
            state['BASE3_RUN_FL'] = False
            inning_end = True
        else:
            inning_end = False

        if inning_end:
            if state['BAT_HOME_ID'] == 1:
                state['INN_CT'] += 1

            state['BAT_HOME_ID'] = 1 - state['BAT_HOME_ID']

            # 9회까지 했을 때
            if state['INN_CT'] > 9:
                state['END_FL'] = True

    away_scores[case_num] = state['AWAY_SCORE_CT']
    home_scores[case_num] = state['HOME_SCORE_CT']

In [16]:
def proc_func(away_scores, home_scores, begin, end):
    for case_num in range(begin, end):
        simulation(away_scores, home_scores, case_num)

In [17]:
f = open("log.txt", "w")  # 로그 파일 열기

# 게임이 시작하는 indices.
start_indices = list(test.index[test['GAME_NEW_FL'] == 'T']) + [test.index[-1] + 1]

trues, total = 0, 0  # total: 무승부를 제외한 모든 경기.

for i in trange(len(start_indices)-1):
    this_game = test.loc[start_indices[i]:start_indices[i+1]-1]
    
    away_team = this_game.iloc[0]['AWAY_TEAM_ID']
    home_team = this_game.iloc[0]['HOME_TEAM_ID']
    
    away_batters = list(this_game['BAT_ID'][this_game['BAT_HOME_ID'] == 0].head(9))
    home_batters = list(this_game['BAT_ID'][this_game['BAT_HOME_ID'] == 1].head(9))

    home_pitcher = this_game['PIT_ID'][this_game['BAT_HOME_ID'] == 0].iloc[0]
    away_pitcher = this_game['PIT_ID'][this_game['BAT_HOME_ID'] == 1].iloc[0]
    
    with Manager() as manager:
        away_scores = manager.list(repeat(0, n_simuls))
        home_scores = manager.list(repeat(0, n_simuls))

        procs = []
        for procID in range(n_procs):  # Simulation using multi-processing.
            begin = procID * (n_simuls // n_procs)
            end = (procID + 1) * (n_simuls // n_procs) if procID < n_procs - 1 else n_simuls
            proc = Process(target=proc_func, args=(away_scores, home_scores, begin, end))
            proc.start()
            procs.append(proc)

        for proc in procs:  # Wait for all processes to complete.
            proc.join()
            
        away_scores = list(away_scores)
        home_scores = list(home_scores)
        
    away_cnt, draw_cnt, home_cnt = 0, 0, 0
    for away_score in away_scores:
        for home_score in home_scores:
            if away_score > home_score:
                away_cnt += 1
            elif away_score < home_score:
                home_cnt += 1
            else:
                draw_cnt += 1
            
    # 예상 결과
    if away_cnt > home_cnt:
        pred = 'away'
    else:
        pred = 'home'
        
    # 실제 결과
    if this_game.iloc[-1]['AWAY_SCORE_CT'] > this_game.iloc[-1]['HOME_SCORE_CT']:
        real = 'away'
    elif this_game.iloc[-1]['AWAY_SCORE_CT'] < this_game.iloc[-1]['HOME_SCORE_CT']:
        real = 'home'
    else:
        real = 'draw'
        
    # 맞은 것 카운트
    if real != 'draw':
        total += 1
    if pred == real:
        trues += 1
    
    # 결과 출력
    f.write(f"실제-{this_game.iloc[-1]['AWAY_SCORE_CT']}:{this_game.iloc[-1]['HOME_SCORE_CT']} / ")
    f.write(f'예상-{np.mean(away_scores)}:{np.mean(home_scores)} / ')
    f.write(f'away:draw:home-{away_cnt/(n_simuls**2)}:{draw_cnt/(n_simuls**2)}:{home_cnt/(n_simuls**2)} / ')
    f.write(f'예측-{pred}')
    if (i+1) % 10 == 0:
        f.write(f' / {i+1}번 게임까지 정확도-{trues / total}')
    f.write('\n')
    f.flush()
    
    # 타자, 투수 정보 추가
    for idx, row in this_game.iterrows():
        bat_id, pit_id = row['BAT_ID'], row['PIT_ID']
        event = row['EVENT_CD']
        
        if bat_id not in bat:
            bat[bat_id] = {key: 0 for key in pa_events + ['pa']}
        if pit_id not in pit:
            pit[pit_id] = {key: 0 for key in pa_events + ['pa']}
            
        bat[bat_id]['pa'] += 1
        bat[bat_id][event] += 1
        bat['average']['pa'] += 1
        bat['average'][event] += 1
        
        pit[pit_id]['pa'] += 1
        pit[pit_id][event] += 1
        pit['average']['pa'] += 1
        pit['average'][event] += 1
        
        if row['BAT_HOME_ID'] == 0:
            team_info[row['HOME_TEAM_ID']]['pa'] += 1
            team_info[row['HOME_TEAM_ID']][event] += 1
        else:
            team_info[row['AWAY_TEAM_ID']]['pa'] += 1
            team_info[row['AWAY_TEAM_ID']][event] += 1
        
f.close()  # 로그 파일 닫기

100%|█████████▉| 1216/1217 [2:01:17<00:05,  5.73s/it] 

IndexError: list index out of range

In [18]:
trues / total

0.5741239892183289